# Bollinger Bands
This strategy combines the moving average strategy 4.9 and the volatility sized intervals strategy 4.10. We will provide liquidity inside the Bollinger Bands. These bands are made up of a lower band BOLL = pa −c·v and an upper band BOLU = pa +c·v. The liquidity position will be [pa −v·c,pa +v·c]. This strategy also has an unused leftover token.

Import code dependencies

In [10]:
import math
from datetime import date, timedelta
from typing import Dict
from _decimal import Decimal

import pandas as pd

from demeter import (
    TokenInfo,
    Actuator,
    Strategy,
    ChainType,
    PeriodTrigger,
    realized_volatility,
    simple_moving_average,
    MarketInfo,
    EvaluatorEnum,
    RowData,
)
from demeter.uniswap import UniV3Pool, UniLpMarket

Set pandas output format and const variables.

In [11]:
pd.options.display.max_columns = None
pd.set_option("display.width", 5000)
c = 2

Custom Add By Volatility strategy with code to add liquidity price at around one day bollinger bands.

In [12]:
class AddByVolatilityStrategy(Strategy):
    """
    We will provide liquidity inside the Bollinger Bands.
    These bands are made up of a lower band BOLL = pa − c · v
    and an upper band BOLU = pa + c · v.
    The liquidity get_position will be [pa − v · c, pa + v · c].

    * pa is simple moving average
    * c is a constant value, =2
    * v is volatility

    we will adjust liquidity every 4 hours, by remove all the liquidity, then even split all the capital into two assets,
    and provide liquidity by the rules above.

    """

    def initialize(self):
        self.add_column(market_key, "sma_1_day", simple_moving_average(self.data[market_key].price, timedelta(days=1)))
        self.add_column(market_key, "volatility", realized_volatility(self.data[market_key].price, timedelta(days=1), timedelta(days=1)))
        self.triggers.append(PeriodTrigger(time_delta=timedelta(hours=4), trigger_immediately=True, do=self.work))
        self.markets.default.even_rebalance(self.data[market_key].price[0])

    def work(self, row_data: RowData):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        lp_row_data = row_data.market_status[market_key]
        if len(lp_market.positions) > 0:
            lp_market.remove_all_liquidity()
            lp_market.even_rebalance(row_data.prices[eth.name])
        if math.isnan(lp_row_data.volatility):
            return
        limit = c * float(row_data.prices[eth.name]) * lp_row_data.volatility
        lp_market.add_liquidity(lp_row_data.sma_1_day - limit, lp_row_data.sma_1_day + limit)


Main logic to run Actuator, init two token and market with key "market1"

In [13]:
usdc = TokenInfo(name="usdc", decimal=6)  # declare  token0
eth = TokenInfo(name="eth", decimal=18)  # declare token1
pool = UniV3Pool(usdc, eth, 0.05, usdc)  # declare pool
market_key = MarketInfo("market1")

actuator = Actuator()  # declare actuator
broker = actuator.broker
market = UniLpMarket(market_key, pool)

broker.add_market(market)
broker.set_balance(usdc, 5000)
broker.set_balance(eth, 0)

actuator.strategy = AddByVolatilityStrategy()

market.data_path = "../data"
market.load_data(ChainType.polygon.name, "0x45dda9cb7c25131df268515131f647d726f50608", date(2023, 8, 13), date(2023, 8, 17))
actuator.set_price(market.get_price_from_data())
# actuator.run()  # run test

2023-11-03 10:24:30,171 - INFO - start load files from 2023-08-13 to 2023-08-17...
2023-11-03 10:24:30,345 - INFO - load file complete, preparing...
2023-11-03 10:24:30,762 - INFO - data has been prepared


Run actuator with evaluators and save result to files

In [14]:
actuator.run(evaluator=[
    EvaluatorEnum.max_draw_down,
    EvaluatorEnum.annualized_returns,
    EvaluatorEnum.net_value,
    EvaluatorEnum.profit,
    EvaluatorEnum.net_value_up_down_rate,
    EvaluatorEnum.eth_up_down_rate,
    EvaluatorEnum.position_fee_profit,
    EvaluatorEnum.position_fee_annualized_returns,
    EvaluatorEnum.position_market_time_rate,
])
evaluating_result: Dict[EvaluatorEnum, Decimal] = actuator.evaluating_indicator

actuator.save_result(
    path="./result",  # save path
    account=True,  # save account status list as a csv file
    actions=True,  # save actions as a json file and a pickle file
)

2023-11-03 10:24:30,810 - INFO - init strategy...
2023-11-03 10:24:30,974 - INFO - start main loop...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 7200/7200 [00:04<00:00, 1625.78it/s]
2023-11-03 10:24:35,411 - INFO - main loop finished
2023-11-03 10:24:35,464 - INFO - Start calculate evaluating indicator...
/Users/florije/PycharmProjects/demeter/demeter/core/evaluating_indicator.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fee_df.sort_values(by=["market1_base_uncollected", "market1_quote_uncollected"], ascending=[False, False], inplace=True)
2023-11-03 10:24:35,526 - INFO - Evaluating indicator has finished it's job.
2023-11-03 10:24:35,527 - INFO - Print actuator summary
2023-11-03 10:24:35,540 - INFO - Backtesting finished, 

Final account status                              
Token balance in broker       
USDC      :2426.887083367097715269579559ETH       :0                        
Position value in markets     
market1(UniLpMarket)
token0    :USDC                     token1    :ETH                      fee       :0.0500                   is 0 base :True                     
positions 
   lower_tick  upper_tick pending0 pending1         liquidity
0      201205      201695        0        0  2409456952156536

Account balance history                           
                                         net_value                           USDC                            ETH              market1_net_value market1_base_uncollected market1_quote_uncollected market1_base_in_position     market1_quote_in_position  market1_position_count
2023-08-13 00:00:00  4998.750000000000000000000000  2498.750000000000000000000000  1.352722809207831809288437799                          0E-24                        0               

['./result/backtest-20231103-102435.account.csv',
 './result/backtest-20231103-102435.action.json',
 './result/backtest-20231103-102435.action.pkl']